In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

from elasticsearch import Elasticsearch
from datasets import load_dataset


from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.document_loaders.pdf import PyPDFDirectoryLoader

from elasticsearch import helpers
from tqdm import tqdm

# Load single PDF file
def load_single_pdf(file_path):
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    return pages

# Load multiple PDF files from directory
def load_pdf_directory(directory_path):
    loader = PyPDFDirectoryLoader(directory_path)
    pages = loader.load()
    return pages



In [1]:
pdf_paths = "../data/pdf/"

In [4]:
pdf_data = load_pdf_directory(pdf_paths)

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # PDF 문서는 더 작은 청크로 나누는 것이 좋음
    chunk_overlap=50, # 청크 간 중복도 줄임
    length_function=len,
    separators=["\n\n", "\n", ".", "!", "?", ";", ":", " ", ""],  # PDF 문서의 구조를 고려한 구분자 추가
    is_separator_regex=False
)

chunks = text_splitter.split_documents(pdf_data)

In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name="../ai_models/base_models/BGE-m3-ko",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
es = Elasticsearch("http://127.0.0.1:9200")


In [7]:
es.options(ignore_status=400).indices.create(
    index="pdf",
    mappings={
        "properties": {
            "content": {"type": "text"},
            "embedding": {
                "type": "dense_vector",
                "dims": 1024,  # 모델에 따라 조정
                "index": True,
                "similarity": "cosine"
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'pdf'})

In [12]:
actions = []
for chunk in tqdm(chunks):
    text = chunk.page_content
    vector = embeddings.embed_documents([text])[0]  # ✅ 문서용

    actions.append({
        "_index": "pdf",
        "_source": {
            "content": text,
            "embedding": vector
        }
    })

helpers.bulk(es, actions)

 49%|████▉     | 6664/13590 [1:03:47<59:15,  1.95it/s]  

In [27]:
query = "피지컬컬 AI 설명"
query_vector = embeddings.embed_query(query)

response = es.search(
    index="pdf",
    knn={
        "field": "embedding",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 20
    }
)

# 결과 출력
for hit in response["hits"]["hits"]:
    print(f"점수: {hit['_score']:.4f}")
    print(f"문서 내용: {hit['_source']['content']}\n")


점수: 0.7662
문서 내용: EU GDPR 이해하기유럽 일반 개인정보보호법General Data Protection  Regulation1. GDPR이란 ?   2018년 5월 25일부터 시행되는 EU(유럽연합)의 개인정보보호 법령이며, 동 법령 위반시 과징금 등 행정처분이 부과될 수 있어 EU와 거래하는 우리나라 기업도 이 법에 위반되지 않도록 주의할 필요가 있습니다.2. GDPR 시행에 따른 주요 변화

점수: 0.7479
문서 내용: GDPR은 DPO가 아니라 컨트롤러 또는 프로세서가 GDPR을 준수하여 개인정보를 처
리하였다는 것을 보장하고, 이를 입증할 수 있는 적절한 기술적·관리적 조치를 이행하
여야 한다고 규정하고 있다. 5 4 즉, GDPR 준수는 컨트롤러나 프로세서의 책임이다.
■
5.6 DPO의 책임 여부
GDPR 관련 규정
■  제37조(DPO의 지정)
■  제38조(DPO의 지위)
■  제39조(DPO의 업무)
■ 전문 제97항
■  Guidelines on Data Protection Officers(DPO) 2016, 
WP29(EDPB 승인)
■ 제31조(개인정보 보호책임자의 지정)한국 개인정보보호법 관련 규정
54   제29조 작업반, The Guidelines on Data Protection Officer, 2017. 04. 05., p.4.

점수: 0.7413
문서 내용: 우리나라에서 GDPR 가이드북을 발간하게 된 배경
2016년 5월 유럽연합(이하 ‘EU’)에서 제정한 「일반 개인정보보호법(General Data Protection 
Regulation)」(이하 ‘GDPR’)이 2018년 5월 25일부터 시행되었습니다.
GDPR은 기존의 EU 개인정보보호 지침인 「1995년 개인정보보호 지침(Data Protection Directive 
95/46/EC)」을 대체하며 보다 강력한 제재를 규정하고 있습니다.
이에 한국인터넷진흥원은 『우리 기업을 위한 유럽 일반 개인정보보호법(GDPR) 안내서』(2017.04.)
와 『